In [ ]:
#@title Install poe-api-wrapper and pyngrok
!pip install -U 'poe-api-wrapper[llm]' pyngrok # [llm] for the OpenAI proxy features
!pip install nest_asyncio # Needed for running asyncio in Colab/Jupyter
!pip install uvicorn # Explicitly install uvicorn

In [ ]:
#@title Install Node.js, nvm, and n8n

# --- Remove previous Node.js/npm attempts (best effort) ---
print("Attempting to clean up previous Node.js/npm installations (best effort)...")
!sudo apt-get purge -y nodejs npm yarn > /dev/null
!sudo rm -rf /usr/local/bin/npm /usr/local/share/man/man1/node* /usr/local/lib/dtrace/node.d /usr/local/lib/node_modules /usr/bin/node /usr/local/bin/node /opt/nodejs* ~/.npm ~/.nvm

# --- Install nvm (Node Version Manager) ---
print("\nInstalling nvm...")
!curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.39.7/install.sh | bash

# --- Source nvm script to make it available in the current shell session ---
print("\nSourcing nvm script (effects are mainly for subsequent commands in this cell via bash -i -c)...")
# Note: Sourcing here primarily helps verify nvm's own installation steps immediately.
# For running nvm commands later, we still need to source it within each `bash -i -c` block.
!export NVM_DIR="$HOME/.nvm" && \
 [ -s "$NVM_DIR/nvm.sh" ] && \. "$NVM_DIR/nvm.sh" && \
 [ -s "$NVM_DIR/bash_completion" ] && \. "$NVM_DIR/bash_completion"

# --- Install desired Node.js version using nvm and set it as default ---
NODE_VERSION_TO_INSTALL="20"
print(f"\nInstalling Node.js v{NODE_VERSION_TO_INSTALL} using nvm and setting it as default...")
!bash -i -c 'export NVM_DIR="$HOME/.nvm" && \
             [ -s "$NVM_DIR/nvm.sh" ] && \. "$NVM_DIR/nvm.sh"; \
             nvm install '"$NODE_VERSION_TO_INSTALL"' && \
             nvm alias default '"$NODE_VERSION_TO_INSTALL"' && \
             nvm use default'

print("\nNode.js installation via nvm attempt complete.")
print("Verifying Node.js and npm versions (should be the version installed by nvm):")
!bash -i -c 'export NVM_DIR="$HOME/.nvm" && \
             [ -s "$NVM_DIR/nvm.sh" ] && \. "$NVM_DIR/nvm.sh"; \
             node -v && npm -v'

# --- Install n8n globally using the nvm-managed Node.js/npm ---
print("\nInstalling n8n globally... This might take a few minutes.")
!bash -i -c 'export NVM_DIR="$HOME/.nvm" && \
             [ -s "$NVM_DIR/nvm.sh" ] && \. "$NVM_DIR/nvm.sh"; \
             npm install -g n8n --unsafe-perm' # --unsafe-perm can help with permission issues in some environments

print("\nn8n installation attempt complete.")
print("Verifying n8n installation:")
!bash -i -c 'export NVM_DIR="$HOME/.nvm" && \
             [ -s "$NVM_DIR/nvm.sh" ] && \. "$NVM_DIR/nvm.sh"; \
             n8n --version'

print("\n--- Setup for n8n with nvm complete ---")
print("If n8n --version shows a version number, you can proceed to the next cell to run both servers.")

In [ ]:
#@title Combined Server Startup (Poe API + n8n)

import os
import json
import subprocess
import threading
import time
import nest_asyncio
from pyngrok import ngrok, conf
import shutil # For rmtree

nest_asyncio.apply()

# --- IMPORTANT: SET YOUR TOKENS HERE ---
POE_P_B_TOKEN =
POE_P_LAT_TOKEN =
NGROK_AUTH_TOKEN =

# --- Poe API Configuration ---
POE_API_PORT = 8000
REPO_ROOT_DIR = "/content/poe-api-wrapper"
POE_API_NGROK_URL = None
poe_api_process = None

# --- n8n Configuration ---
N8N_PORT = 5678
N8N_NGROK_URL = None
n8n_process = None

# --- Warnings ---
if POE_P_B_TOKEN == "YOUR_P-B_COOKIE_HERE" or POE_P_LAT_TOKEN == "YOUR_P-LAT_COOKIE_HERE":
    print("⚠️ PLEASE SET YOUR POE TOKENS in the code above!")
if NGROK_AUTH_TOKEN == "YOUR_NGROK_AUTHTOKEN_HERE" or not NGROK_AUTH_TOKEN:
    print("⚠️ PLEASE SET YOUR NGROK AUTH TOKEN in the code above!")
print("ℹ️ Reminder: The poe-api-wrapper GitHub repository was archived on Mar 10, 2025, and is now read-only. [1]")

# --- Poe API Setup (Clone repo and create secrets.json - assuming this part is fine) ---
if not os.path.exists(REPO_ROOT_DIR):
    print(f"Cloning poe-api-wrapper repository to {REPO_ROOT_DIR}...")
    clone_process = subprocess.run(['git', 'clone', 'https://github.com/snowby666/poe-api-wrapper.git', REPO_ROOT_DIR], capture_output=True, text=True)
    if clone_process.returncode == 0:
        print("Poe-api-wrapper repository cloned successfully.")
    else:
        print(f"Error cloning repository: {clone_process.stderr}")
else:
    print(f"Poe-api-wrapper repository already exists at {REPO_ROOT_DIR}.")

secrets_data = {"tokens": [{"p-b": POE_P_B_TOKEN, "p-lat": POE_P_LAT_TOKEN}], "proxy": None, "TIMEOUT": 20}
openai_package_dir = os.path.join(REPO_ROOT_DIR, "poe_api_wrapper", "openai")
secrets_file_path = os.path.join(openai_package_dir, "secrets.json")
os.makedirs(openai_package_dir, exist_ok=True)
with open(secrets_file_path, "w") as f:
    json.dump(secrets_data, f, indent=4)
print(f"Created/Updated secrets.json for Poe API at: {secrets_file_path}")

# --- Function to log subprocess output ---
def log_pipe(pipe, prefix):
    try:
        for line in iter(pipe.readline, ''): print(f"[{prefix}] {line.strip()}")
    except Exception as e: print(f"Error reading from {prefix} pipe: {e}")
    finally:
        if hasattr(pipe, 'close'): pipe.close()

# --- Server startup functions (start_poe_api_server, start_n8n_server) remain the same ---
def start_poe_api_server():
    global poe_api_process
    app_module_path = "poe_api_wrapper.openai.api:app"
    print(f"Attempting to start Poe API Wrapper with Uvicorn ({app_module_path}) on port {POE_API_PORT}...")
    poe_api_process = subprocess.Popen(
        ["uvicorn", app_module_path, "--host", "0.0.0.0", "--port", str(POE_API_PORT)],
        cwd=REPO_ROOT_DIR, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    print(f"Poe API (Uvicorn) process initiated with PID: {poe_api_process.pid}")
    threading.Thread(target=log_pipe, args=(poe_api_process.stdout, "POE_API_STDOUT"), daemon=True).start()
    threading.Thread(target=log_pipe, args=(poe_api_process.stderr, "POE_API_STDERR"), daemon=True).start()

def start_n8n_server():
    global n8n_process
    print(f"Attempting to start n8n server (port {N8N_PORT} via env var) using nvm-managed Node.js...")
    n8n_command = f"""
    export NVM_DIR="$HOME/.nvm" && [ -s "$NVM_DIR/nvm.sh" ] && . "$NVM_DIR/nvm.sh" && \
    [ -s "$NVM_DIR/bash_completion" ] && . "$NVM_DIR/bash_completion"; \
    export N8N_PORT={N8N_PORT} && n8n start"""
    n8n_process = subprocess.Popen(
        ["bash", "-i", "-c", n8n_command],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    print(f"n8n process initiated with PID: {n8n_process.pid}")
    threading.Thread(target=log_pipe, args=(n8n_process.stdout, "N8N_STDOUT"), daemon=True).start()
    threading.Thread(target=log_pipe, args=(n8n_process.stderr, "N8N_STDERR"), daemon=True).start()

# --- Main Execution Block ---
try:
    # --- AGGRESSIVE NGROK CLEANUP ---
    print("\n--- Performing aggressive ngrok cleanup ---")
    # 1. Kill any running ngrok processes (system-wide)
    print("Attempting to kill any existing ngrok processes (system-wide)...")
    try:
        subprocess.run("pkill -f ngrok || true", shell=True, check=False, timeout=5) # More robust kill
        print("System-wide ngrok process kill attempt complete.")
    except subprocess.TimeoutExpired:
        print("System-wide ngrok process kill attempt timed out.")
    except Exception as e_pkill:
        print(f"Error during pkill ngrok: {e_pkill}")
    time.sleep(2)

    # 2. If pyngrok's ngrok process object exists, try to kill it specifically
    if ngrok.get_ngrok_process(): # This checks pyngrok's managed process
        print("Killing ngrok process known to pyngrok...")
        try:
            ngrok.kill()
            time.sleep(2)
        except Exception as e_kill_pyngrok:
            print(f"Note: Error during pyngrok.kill(): {e_kill_pyngrok}")

    # 3. Disconnect any tunnels pyngrok is aware of
    try:
        tunnels = ngrok.get_tunnels()
        if tunnels:
            print("Disconnecting tunnels known to pyngrok...")
            for tunnel in tunnels:
                ngrok.disconnect(tunnel.public_url)
            time.sleep(1)
    except Exception as e_disconnect:
        print(f"Note: Error during pyngrok.get_tunnels/disconnect(): {e_disconnect}")

    # 4. Remove pyngrok's default binary cache and ngrok's default config locations
    home_dir = os.path.expanduser("~")
    paths_to_remove = [
        os.path.join(home_dir, ".pyngrok"), # This holds cached binary and default config for pyngrok
        os.path.join(home_dir, ".ngrok2"),  # Older ngrok CLI config
        os.path.join(home_dir, ".config", "ngrok") # Newer ngrok CLI config
    ]
    for path_to_remove in paths_to_remove:
        print(f"Attempting to remove: {path_to_remove}")
        if os.path.exists(path_to_remove):
            try:
                if os.path.isdir(path_to_remove):
                    shutil.rmtree(path_to_remove)
                else:
                    os.remove(path_to_remove)
                print(f"Removed: {path_to_remove}")
            except Exception as e_rm:
                print(f"Error removing {path_to_remove}: {e_rm}")
    print("--- Aggressive ngrok cleanup attempt complete ---")

    # --- Configure ngrok (once for all tunnels) ---
    if NGROK_AUTH_TOKEN and NGROK_AUTH_TOKEN != "YOUR_NGROK_AUTHTOKEN_HERE":
        try:
            # pyngrok will now be forced to download a fresh ngrok binary if its cache was cleared.
            print(f"\nSetting ngrok authtoken via pyngrok: {NGROK_AUTH_TOKEN}")
            ngrok.set_auth_token(NGROK_AUTH_TOKEN)
            print("ngrok.set_auth_token called successfully.")

            # Optional: Verify ngrok binary path pyngrok will use
            # print(f"pyngrok is configured to use ngrok binary at: {conf.get_default().ngrok_path}")
            # print(f"pyngrok is configured to use config file at: {conf.get_default().config_path}")

        except Exception as e:
            print(f"Error during ngrok setup/authentication (set_auth_token phase): {e}")
            raise
    else:
        print("Skipping ngrok authentication as token is not set (may lead to limitations).")
        if not NGROK_AUTH_TOKEN: # Explicitly raise if token is empty
             raise ValueError("NGROK_AUTH_TOKEN is empty. Please set it.")


    # --- Start Poe API Server and its ngrok tunnel ---
    print("\nPreparing to start Poe API server thread...")
    poe_server_thread = threading.Thread(target=start_poe_api_server, daemon=True)
    poe_server_thread.start()
    print("Poe API server thread started. Waiting for Poe API to initialize (approx 20-30s)...")
    time.sleep(30)

    if poe_api_process and poe_api_process.poll() is None:
        print(f"Starting ngrok tunnel for Poe API on localhost:{POE_API_PORT}...")
        try:
            poe_public_url_obj = ngrok.connect(POE_API_PORT, "http", name="poe-api")
            POE_API_NGROK_URL = str(poe_public_url_obj)
            print(f"✅ Poe API Wrapper should be running and exposed at: {POE_API_NGROK_URL}")
        except Exception as e:
            print(f"❌ Error starting ngrok for Poe API: {e}")
            print("   Check POE_API STDERR logs and ngrok authentication status.")
    else:
        print("❌ Poe API server failed to start or terminated prematurely. Skipping its ngrok tunnel.")

    # --- Start n8n Server and its ngrok tunnel ---
    print("\nPreparing to start n8n server thread...")
    n8n_server_thread = threading.Thread(target=start_n8n_server, daemon=True)
    n8n_server_thread.start()
    print("n8n server thread started. Waiting for n8n to initialize (approx 45-75s)...")
    time.sleep(75)

    if n8n_process and n8n_process.poll() is None:
        print(f"Starting ngrok tunnel for n8n on localhost:{N8N_PORT}...")
        try:
            n8n_public_url_obj = ngrok.connect(N8N_PORT, "http", name="n8n")
            N8N_NGROK_URL = str(n8n_public_url_obj)
            print(f"✅ n8n should be running and exposed at: {N8N_NGROK_URL}")
            print(f"   You might need to set up an admin user on your first n8n visit.")
        except Exception as e:
            print(f"❌ Error starting ngrok for n8n: {e}")
            print("   Check N8N STDERR/STDOUT logs and ngrok authentication status.")
    else:
        print("❌ n8n server failed to start or terminated prematurely. Skipping its ngrok tunnel.")

    # --- Keep this cell running ---
    # (The rest of your keep-alive and finally block remains the same)
    if POE_API_NGROK_URL or N8N_NGROK_URL:
        print(f"\n--- Servers and ngrok tunnels should be active ---")
        if POE_API_NGROK_URL: print(f"Poe API Public URL: {POE_API_NGROK_URL}")
        if N8N_NGROK_URL: print(f"n8n Public URL: {N8N_NGROK_URL}")
        print("\nInterrupt this cell (click the stop button) to shut down all services.")
        while True:
            time.sleep(30)
            print(".", end="", flush=True)
            poe_terminated = poe_api_process and poe_api_process.poll() is not None
            n8n_terminated = n8n_process and n8n_process.poll() is not None

            if poe_terminated and POE_API_NGROK_URL:
                print(f"\n[WARNING] Poe API (Uvicorn) process (PID {poe_api_process.pid if poe_api_process else 'N/A'}) terminated unexpectedly with code {poe_api_process.returncode if poe_api_process else 'N/A'}.")
            if n8n_terminated and N8N_NGROK_URL:
                print(f"\n[WARNING] n8n process (PID {n8n_process.pid if n8n_process else 'N/A'}) terminated unexpectedly with code {n8n_process.returncode if n8n_process else 'N/A'}.")

            should_poe_be_running = POE_API_NGROK_URL is not None
            should_n8n_be_running = N8N_NGROK_URL is not None
            poe_done = not should_poe_be_running or poe_terminated
            n8n_done = not should_n8n_be_running or n8n_terminated

            if poe_done and n8n_done and (should_poe_be_running or should_n8n_be_running):
                 print("\nAll active server processes appear to have terminated.")
                 break
            elif not (should_poe_be_running or should_n8n_be_running) and \
                 (not poe_api_process or poe_api_process.poll() is not None) and \
                 (not n8n_process or n8n_process.poll() is not None):
                 print("\nNeither server process appears to have started successfully with a tunnel. Exiting keep-alive loop.")
                 break
    else:
        print("\n--- Setup failed. Neither Poe API nor n8n started with an ngrok tunnel. Check logs. ---")

except KeyboardInterrupt:
    print("\nCell execution interrupted by user. Shutting down all services...")
except Exception as e:
    print(f"\nAn unexpected error occurred: {e}. Shutting down...")
finally:
    print("\n--- Initiating shutdown sequence ---")
    if POE_API_NGROK_URL:
        try:
            print(f"Disconnecting Poe API ngrok tunnel: {POE_API_NGROK_URL}")
            ngrok.disconnect(POE_API_NGROK_URL)
        except Exception as e_ngrok_disc_poe: print(f"Error disconnecting Poe API ngrok tunnel: {e_ngrok_disc_poe}")
    if N8N_NGROK_URL:
        try:
            print(f"Disconnecting n8n ngrok tunnel: {N8N_NGROK_URL}")
            ngrok.disconnect(N8N_NGROK_URL)
        except Exception as e_ngrok_disc_n8n: print(f"Error disconnecting n8n ngrok tunnel: {e_ngrok_disc_n8n}")
    if poe_api_process and poe_api_process.poll() is None:
        print(f"Terminating Poe API (Uvicorn) process (PID {poe_api_process.pid})...")
        poe_api_process.terminate()
        try: poe_api_process.wait(timeout=5)
        except subprocess.TimeoutExpired: poe_api_process.kill()
        print("Poe API (Uvicorn) process stopped.")
    if n8n_process and n8n_process.poll() is None:
        print(f"Terminating n8n process (PID {n8n_process.pid})...")
        n8n_process.terminate()
        try: n8n_process.wait(timeout=5)
        except subprocess.TimeoutExpired: n8n_process.kill()
        print("n8n process stopped.")
    try:
        if ngrok.get_ngrok_process():
            print("Killing main ngrok process (if still active)...")
            ngrok.kill()
            print("Main ngrok process killed.")
    except Exception as e_ngrok_kill: print(f"Notice: Error killing main ngrok process: {e_ngrok_kill}")
    print("--- Shutdown complete ---")